In [21]:
# Importar librerías necesarias
import pandas as pd
from langdetect import detect
from bs4 import BeautifulSoup
from transformers import BertTokenizer, BertForSequenceClassification, Trainer, TrainingArguments, BertConfig
from sklearn.model_selection import train_test_split
from sklearn.metrics import precision_recall_fscore_support, accuracy_score, confusion_matrix, ConfusionMatrixDisplay
from torch.utils.data import Dataset, DataLoader
import torch
import re
import numpy as np



In [22]:
import torch
print(torch.__version__)


2.4.1+cpu


In [23]:
# 1. Cargar el archivo CSV con los datos
try:
    df = pd.read_csv('Abstracts_filtrados.csv')
    print("Archivo 'Abstracts_filtrados.csv' cargado exitosamente.")
except FileNotFoundError:
    print("Archivo 'Abstracts_filtrados.csv' no encontrado. Asegúrate de que está en el directorio de trabajo.")



Archivo 'Abstracts_filtrados.csv' cargado exitosamente.


In [24]:
# 2. Función para limpiar el texto
def limpiar_texto(texto):
    texto_limpio = BeautifulSoup(texto, "html.parser").get_text()  # Eliminar etiquetas HTML
    texto_limpio = re.sub(r'\s+', ' ', texto_limpio)  # Eliminar múltiples espacios
    texto_limpio = re.sub(r'[^a-zA-Z0-9\s]', '', texto_limpio)  # Eliminar caracteres especiales
    if detect(texto_limpio) == 'en':  # Verificar si el texto está en inglés
        return texto_limpio.strip()
    return ''

# Aplicar la función de limpieza al DataFrame
df['Abstract_Limpio'] = df['Abstract'].apply(limpiar_texto)

# Guardar el DataFrame limpio
df.to_csv('Abstracts_limpios.csv', index=False)

In [25]:
# 3. Cargar el tokenizador de BERT
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

# Dividir el dataset en entrenamiento y evaluación
X = df['Abstract_Limpio'].tolist()
y = df['ODS'].tolist()

# Convertir etiquetas a valores numéricos (si es necesario)
# Asumiendo que y es una lista de strings con las clases de ODS
unique_labels = list(set(y))
label_to_id = {label: idx for idx, label in enumerate(unique_labels)}
y = [label_to_id[label] for label in y]

# Dividir en conjunto de entrenamiento y evaluación
X_train, X_eval, y_train, y_eval = train_test_split(X, y, test_size=0.2, random_state=42)

# Tokenizar los textos para entrenamiento y evaluación
train_encodings = tokenizer(X_train, truncation=True, padding=True, max_length=512)
eval_encodings = tokenizer(X_eval, truncation=True, padding=True, max_length=512)

c:\Users\Jhonr\anaconda3\envs\NPL_ODS\lib\site-packages\transformers\tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [26]:
# 4. Crear el Dataset personalizado para PyTorch
class ODSDataset(Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels
    
    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx], dtype=torch.long)
        return item
    
    def __len__(self):
        return len(self.labels)

# Crear los datasets de entrenamiento y evaluación
train_dataset = ODSDataset(train_encodings, y_train)
eval_dataset = ODSDataset(eval_encodings, y_eval)

In [34]:
# 5. Configurar el modelo con el número correcto de etiquetas
num_labels = len(unique_labels)  # Obtener el número de clases únicas
config = BertConfig.from_pretrained('bert-base-uncased')
config.hidden_dropout_prob = 0.3  # Incrementar dropout para regularización
config.num_labels = num_labels  # Ajustar el número de etiquetas

# Cargar el modelo con la configuración ajustada
model = BertForSequenceClassification(config)


In [36]:
# 6. Configurar los argumentos de entrenamiento
training_args = TrainingArguments(
    output_dir='./results',            # Directorio donde se guardan los resultados
    evaluation_strategy="epoch",       # Evalúa el modelo al final de cada época
    per_device_train_batch_size=8,     # Tamaño del batch por dispositivo
    per_device_eval_batch_size=8,      # Tamaño del batch para evaluación
    num_train_epochs=5,                # Número de épocas de entrenamiento
    learning_rate=2e-5,                # Tasa de aprendizaje baja
    weight_decay=0.01,                 # Usar decay para evitar sobreajuste
    logging_dir='./logs',              # Directorio de logs
    logging_steps=10,                  # Loguear cada 10 pasos
    warmup_steps=500                   # Pasos de calentamiento para estabilidad
)

# Verificar el número de clases en las etiquetas
print(f"Total de clases únicas: {num_labels}")
print(f"Etiquetas: {unique_labels}")

Total de clases únicas: 11
Etiquetas: ['\tODS 9 - Industria, Innovación e Infraestructura', 'ODS 7 - Energía Asequible y No Contaminante', 'ODS 2: Hambre Cero', 'ODS 13 - Acción por el Clima', '\tODS 3 - Salud y Bienestar', 'ODS 8 - Trabajo Decente y Crecimiento Económico', '\tODS 4 - Educación de Calidad', 'ODS 3 - Salud y Bienestar', 'ODS 2 - Hambre Cero', 'ODS 6 - Agua Limpia y Saneamiento', 'ODS 9 - Industria, Innovación e Infraestructura']


In [37]:
# 7. Definir una función para calcular métricas personalizadas
def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average='weighted', zero_division=1)
    acc = accuracy_score(labels, preds)
    return {
        'accuracy': acc,
        'precision': precision,
        'recall': recall,
        'f1': f1,
    }

In [38]:
# 8. Inicializar el entrenador con las métricas personalizadas
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    compute_metrics=compute_metrics   # Función personalizada para calcular métricas
)


In [39]:
# 9. Iniciar el entrenamiento
trainer.train()




                                              

  0%|          | 0/30 [05:00<?, ?it/s]       



{'eval_loss': 2.2088963985443115, 'eval_accuracy': 0.5833333333333334, 'eval_precision': 0.7569444444444445, 'eval_recall': 0.5833333333333334, 'eval_f1': 0.4298245614035088, 'eval_runtime': 6.7908, 'eval_samples_per_second': 1.767, 'eval_steps_per_second': 0.295, 'epoch': 1.0}



  0%|          | 0/30 [06:02<?, ?it/s]         

{'loss': 2.3435, 'grad_norm': 15.836101531982422, 'learning_rate': 4.0000000000000003e-07, 'epoch': 1.67}





                                               

  0%|          | 0/30 [06:38<?, ?it/s]       



...


  0%|          | 0/30 [11:17<?, ?it/s]         

{'loss': 2.0471, 'grad_norm': 15.62112045288086, 'learning_rate': 1.2000000000000002e-06, 'epoch': 5.0}





                                               

  0%|          | 0/30 [11:26<?, ?it/s]       


100%|██████████| 30/30 [07:59<00:00, 15.99s/it]

{'eval_loss': 1.7412573099136353, 'eval_accuracy': 0.5833333333333334, 'eval_precision': 0.7569444444444445, 'eval_recall': 0.5833333333333334, 'eval_f1': 0.4298245614035088, 'eval_runtime': 6.6311, 'eval_samples_per_second': 1.81, 'eval_steps_per_second': 0.302, 'epoch': 5.0}
{'train_runtime': 479.592, 'train_samples_per_second': 0.49, 'train_steps_per_second': 0.063, 'train_loss': 2.1885454177856447, 'epoch': 5.0}


TrainOutput(global_step=30, training_loss=2.1885454177856447, metrics={'train_runtime': 479.592, 'train_samples_per_second': 0.49, 'train_steps_per_second': 0.063, 'total_flos': 59058301113990.0, 'train_loss': 2.1885454177856447, 'epoch': 5.0})

In [40]:
# 10. Evaluar el modelo
results = trainer.evaluate()
print("Resultados de evaluación:", results)

100%|██████████| 2/2 [00:02<00:00,  1.04s/it]

Resultados de evaluación: {'eval_loss': 1.7412573099136353, 'eval_accuracy': 0.5833333333333334, 'eval_precision': 0.7569444444444445, 'eval_recall': 0.5833333333333334, 'eval_f1': 0.4298245614035088, 'eval_runtime': 6.6583, 'eval_samples_per_second': 1.802, 'eval_steps_per_second': 0.3, 'epoch': 5.0}


In [41]:
# 11. Obtener las predicciones del conjunto de evaluación
predictions = trainer.predict(eval_dataset)
y_preds = np.argmax(predictions.predictions, axis=1)


100%|██████████| 2/2 [00:02<00:00,  1.07s/it]
